In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [1]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


In [2]:
np.random.seed(0)
translator = CirqTranslater(3, untouchable_blocks = [1])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

circuit_db = concatenate_dbs([u2_layer(translator)]*2)

c, circuit_db = translator.give_circuit(circuit_db)

dd1  =circuit_db.copy()
dd1["block_id"] = 1
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)


dd1  =circuit_db.copy()
dd1["block_id"] = 2
dd1["symbol"] = None
circuit_db = concatenate_dbs([circuit_db, dd1])
c, circuit_db = translator.give_circuit(circuit_db)


simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.apply_relatives_to_first = True
simplifier.relative_rules = [simplifier.rule_1]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 

simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f94262e70f0>>


In [3]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



0.002840115185851185
(0, 0): ───Rz(th_0)────Rx(th_1)────Rz(th_2)────@───Rz(th_6)───X──────────────@───Rz(th_9)────Rx(th_10)───Rz(th_11)───Rz(th_33)───Rx(th_34)───Rz(th_35)───────────────────────────────────────────────────────────────────────────────────────────────────────────────X───Ry(th_37)───@───Ry(th_38)───X───Rz(th_42)───Rx(th_43)───Rz(th_44)───Rz(th_45)───Rx(th_46)───Rz(th_47)───@───Rz(th_51)───X───────────────@───Rz(th_54)───Rx(th_55)───Rz(th_56)───Rz(th_78)───Rx(th_79)───Rz(th_80)───────────────────────────────────────────────────────────────────────────────────────────────────────────────X───Ry(th_82)───@───Ry(th_83)───X───Rz(th_87)───Rx(th_88)───Rz(th_89)───Rz(th_90)────Rx(th_91)────Rz(th_92)────@───Rz(th_96)───X───────────────@───Rz(th_99)────Rx(th_100)───Rz(th_101)───Rz(th_123)───Rx(th_124)───Rz(th_125)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───Ry(th_127)───@───Ry(th_128)───X───Rz(th_132)───

                                                                                                                                                                                                    ┌──────────┐                                                                                                                                                                                                                                       ┌──────────┐                           ┌──────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [4]:
len(simplified_db)

342

In [5]:
len(circuit_db)

432